In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np
import math
import time
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt 
class Functions(nn.Module):
    def __init__(self):
        super(Functions, self).__init__()

    def forward(self, x):
        well1 = -torch.exp(-3 * ((x[0] - 0.5)**2 + x[1]**2))
        well2 = -torch.exp(-3 * ((x[0] + 0.5)**2 + (x[1] + 0.5)**2))
        bump1 = torch.exp(-15 * ((x[0] + 0.2)**2 + (x[1] + 0.4)**2))
        bump2 = torch.exp(-15 * ((x[0] + 0.0)**2 + (x[1] - 0.3)**2))
        z=well1 + well2 + bump1 + bump2+1.0190055296306975
        return z.unsqueeze(0)

class FourierPathNN(nn.Module):
    def __init__(self,x1,x2, num_terms=10):
        super(FourierPathNN, self).__init__()
        self.num_terms = num_terms
        
        
        self.register_buffer('x1', x1.flatten())  # Store as a constant buffer
        self.register_buffer('x2', x2.flatten())  # Store as a constant buffer
        
        # Fourier coefficients for sine terms (b_n terms)
        self.b = nn.Parameter(torch.zeros(num_terms, x1.numel()))  # Learnable sine coefficients

    def forward(self, t_values):
        """
        Generate weights for a given t value (or array of t values).
        """
        t_values = t_values.view(-1, 1)
        a_0 = (1 - t_values) * self.x1 + t_values * self.x2
        sine_terms = torch.zeros_like(a_0)
        for n in range(1, self.num_terms + 1):
            sine_terms += self.b[n-1].view(1, -1) * torch.sin(n * torch.pi * t_values)
        weights = a_0 + sine_terms  # Combine a_0 and sine terms
        return weights.view(-1, *self.x1.shape)
    
def train_fourier_nn(path_model, cnn_model,num_terms, num_steps=300, lr=0.01):
    optimizer = optim.Adam(path_model.parameters(), lr=lr)
    t_values = torch.linspace(0, 1, 101).unsqueeze(1)  # 200 points between 0 and 1
    Min_Loss=None
    for step in range(num_steps):
        optimizer.zero_grad()
        path_weights = path_model(t_values)  # Generate weights dynamically
        loss,Data_array = loss_fn(path_weights, cnn_model,num_terms)
        loss.backward()
        for param in path_model.parameters():
            if param.grad is not None:
                param.grad += torch.randn_like(param.grad) * 1.0
        optimizer.step()
        if Min_Loss is None or Min_Loss > loss.item():
            Min_array=Data_array
            Min_Loss=loss.item()
            Min_path=path_weights
        #print(f"Step {step}/{num_steps}, Loss: {loss:.6f}")
    return Min_array,Min_path
def loss_fn(weights, cnn_model,num_terms):
    """
    Compute the loss by injecting weights into the CNN model.
    """
    t_values = torch.linspace(0, 1, 101).unsqueeze(1)
    #cnn_model.eval()
    total_loss = 0.0
    criterion = nn.MSELoss()
    Data_array=np.zeros(len(weights))
    Loss_sum=0.0
    #print(len(weights))
    smoothness_loss = 0.0
    for j in range(len(weights) - 1):
        smoothness_loss += torch.norm(weights[j+1] - weights[j])**2
    for i in range(len(weights)):
        
        loss = criterion(cnn_model(weights[i]), torch.tensor([0],dtype=torch.float))
        total_loss += loss
        Data_array[i]=loss.item()
    print(total_loss,smoothness_loss)
    return total_loss+35*smoothness_loss,Data_array
# Main Function
def main(x1,x2):
    # Initialize Models
    num_term=15
    fourier_nn = FourierPathNN(x1,x2,num_terms=num_term)

    # Initialize the FunctionalCNN
    cnn = Functions()

    # Train the FourierPathNN
    Min_array,Min_path=train_fourier_nn(fourier_nn, cnn,num_term)
    return Min_path
x1=torch.tensor([-0.62,-0.54])
x2=torch.tensor([0.49,-0.02])
Min_path=main(x1,x2)

In [ ]:
x1=torch.tensor([-0.62,-0.54])
x2=torch.tensor([0.49,-0.02])
t_array=np.linspace(0,1,101)
Straight_Loss_array=np.zeros(len(t_array))
Optimized_Loss_array=np.zeros(len(t_array))
def two_wells_with_bump(x, y):
    well1 = -np.exp(-3*((x-0.5)**2 + y**2))
    well2 = -np.exp(-3*((x+0.5)**2 + (y+0.5)**2))
    bump1 = np.exp(-15*((x+0.2)**2 + (y+0.4)**2))
    bump2 = np.exp(-15*((x+0.0)**2 + (y-0.3)**2))
    return well1 + well2 + bump1+ bump2+1.0190055296306975
x = np.linspace(-1, 1, 200)
y = np.linspace(-1, 1, 200)
X, Y = np.meshgrid(x, y)
Z = two_wells_with_bump(X, Y)
x_vals = np.linspace(x1[0], x2[0], len(t_array))
y_vals = np.linspace(x1[1], x2[1], len(t_array))
for i in range(len(t_array)):
    Straight_Loss_array[i]=two_wells_with_bump(x_vals[i],y_vals[i])
    Optimized_Loss_array[i]=two_wells_with_bump(Min_path.detach().numpy()[i,0],Min_path.detach().numpy()[i,1])
plt.figure(figsize=(8,6))
contours = plt.contour(X, Y, Z, levels=50, cmap='viridis')
plt.colorbar(contours)
plt.plot(x_vals, y_vals, label="Linear Interpolated Line", color='blue',linestyle='--')

plt.title('2D examples')
plt.xlabel('x')
plt.ylabel('y')
plt.plot(Min_path.detach().numpy()[:,0],Min_path.detach().numpy()[:,1],label="Optimized Path")
plt.legend()
plt.xlim([-0.8,0.8])
plt.ylim([-0.75,0.6])
plt.show()
plt.plot(Straight_Loss_array,label="Linear Interpolated Line")
plt.plot(Optimized_Loss_array,label="Optimized Path")
plt.xlabel("t")
plt.ylabel("Loss")
plt.title("Loss along Straight Path vs Optimized Path")
plt.legend()
plt.show()